In [8]:
import os
from rosemary import jpt_setup; jpt_setup()

from transformers.trainer_utils import get_last_checkpoint

import re
from llm.submit import (
    multiline_to_singleline,
    submit_job_ccc,
    submit_job_aimos,
    submit_job,
    get_run_statistics)
import pandas as pd
import json
import platform
import tempfile
import subprocess
import shlex
import datetime
import itertools
import socket

arch = platform.uname().processor
hostname = socket.gethostname()
cluster = 'ccc' if hostname.startswith('ccc') else ('dcs' if hostname.startswith('dcs') else 'npl')
arch, cluster

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


('ppc64le', 'dcs')

In [9]:

shell_scripts_template_slurm = """
echo "Running on $SLURM_JOB_NODELIST"
echo "======"

master_addr=$(scontrol show hostnames "$SLURM_JOB_NODELIST" | head -n 1)
master_port=10002
RDZV_ENDPOINT=$master_addr:$master_port

source ~/.profile
conda activate open-instruct
cd /gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/open-instruct/

set -e
set -x
echo "======"
srun {cmd}

[ ! -f "{log_dir}/$SLURM_JOB_ID*.out" ] && mv {log_dir}/$SLURM_JOB_ID*.out {save_dir}
"""

shell_scripts_template_lsf = """
echo "Running on $LSB_DJOB_HOSTFILE"
echo "======"

master_addr=$(head -n 1 "$LSB_DJOB_HOSTFILE")
master_port=10002
RDZV_ENDPOINT=$master_addr:$master_port

source ~/.profile
conda activate open-instruct
cd /dccstor/mit_fm/wpq/github/mitibm2023/external/open-instruct/

set -e
set -x
echo "======"
srun {cmd}

[ ! -f "{log_dir}/$LSB_JOBID*.out" ] && mv {log_dir}/$LSB_JOBID*.out {save_dir}
"""

shell_scripts_template = shell_scripts_template_slurm \
    if arch == 'ppc64le' else shell_scripts_template_lsf

print(shell_scripts_template)


echo "Running on $SLURM_JOB_NODELIST"
echo "======"

master_addr=$(scontrol show hostnames "$SLURM_JOB_NODELIST" | head -n 1)
master_port=10002
RDZV_ENDPOINT=$master_addr:$master_port

source ~/.profile
conda activate open-instruct
cd /gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/open-instruct/

set -e
set -x
echo "======"
srun {cmd}

[ ! -f "{log_dir}/$SLURM_JOB_ID*.out" ] && mv {log_dir}/$SLURM_JOB_ID*.out {save_dir}



In [10]:
package_dir = "/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi"
cache_dir = '/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache'
preprocessed_data = "/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/data/processed"

envs = {
    "CACHE": cache_dir,
    "DOREMI_DIR": package_dir,
    "PILE_DIR": os.path.join(package_dir, "data", 'raw'),
    "PREPROCESSED_PILE_DIR": preprocessed_data,
    "MODEL_OUTPUT_DIR": os.path.join(package_dir, 'results'),
    "PARTITION": "el8",
    "HF_HOME": cache_dir,
    "TRANSFORMERS_CACHE": cache_dir,
    "HF_DATASETS_CACHE": cache_dir,
    "HF_DATASETS_IN_MEMORY_MAX_SIZE": "0",
    "TORCH_EXTENSIONS_DIR": cache_dir,
    "TMPDIR": cache_dir,
    "WANDB_DIR": os.path.join(cache_dir, "wandb"),
    "PREPROCESSED_DATA": preprocessed_data,
    'PREPROCESSED_CACHE': os.path.join(cache_dir, 'preprocessed_cache', 'perdomain_pile_preprocessed'),

}

for k, v in envs.items():
    os.environ[k] = v
    
os.makedirs(cache_dir, exist_ok=True)
print(';'.join([f'export {k}={v}' for k, v in envs.items()]))

export CACHE=/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache;export DOREMI_DIR=/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi;export PILE_DIR=/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/data/raw;export PREPROCESSED_PILE_DIR=/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/data/processed;export MODEL_OUTPUT_DIR=/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/results;export PARTITION=el8;export HF_HOME=/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache;export TRANSFORMERS_CACHE=/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache;export HF_DATASETS_CACHE=/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache;export HF_DATASETS_IN_MEMORY_MAX_SIZE=0;export TORCH_EXTENSIONS_DIR=/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache;export TMPDIR=/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache;export WANDB_DIR=/gpfs/

In [57]:
# populate domain weight config
import json
domain_config_path = os.path.abspath('../configs/humanmix_baseline_50kvocab.json')
domain_weights = {"cot": .25, "flan_v2": .25, "dolly": .25, "oasst1": .25}
domain_weights = {'cot': 0.5, 'flan_v2': 0.25, 'dolly': 0.12, 'oasst1': 0.13}
abbr_train_file = 'humanmix'

domain_config = {"train_domain_weights": domain_weights, "eval_domain_weights": domain_weights}
with open(domain_config_path, 'w') as f:
    json.dump(domain_config, f)


1562.5

In [66]:
# train gpt2 base model
job_name = 'ft1'

test_run = 1
test_run = bool(test_run)
job_duration = 6

nodes = 1 # 128/(5*6*2)~=2.1
num_cpus, cpu_mem = (144, 512) if arch == 'ppc64le' else (32, 64)
num_gpus = 6; gpu_type = 'v100'
# num_gpus = 1; gpu_type = 'v100'

overwrite_output_dir = True if test_run else False

model_name_or_path = ('/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/results/baselines/'
                      'gpt2-medium')
model_type = 'gpt2'; abbr_model_name = 'gpt2-medium'

cache_dir = envs['CACHE']
domain_config_path = os.path.abspath('../configs/humanmix_baseline_50kvocab.json')
dataset_dir = preprocessed_data


total_batch_size = 128 # # 64*8=512
per_device_train_batch_size = 2
gradient_accumulation_steps = 1
gradient_accumulation_steps = int(total_batch_size/(num_gpus*nodes)/per_device_train_batch_size)
max_steps = int(200000/total_batch_size); save_steps = 10 # 200k steps.
gradient_checkpointing = False

print(f"Training {model_name_or_path} "
      f"using {num_gpus} GPUs, "
      f"{per_device_train_batch_size} batch size per GPU, "
      f"{gradient_accumulation_steps} gradient accumulation steps,"
      f"for {max_steps} max steps.")


if nodes == 1:
    exe = 'python' if num_gpus==1 else \
        f"""torchrun \
        --nproc_per_node={num_gpus} \
        --master_port=10002"""
else:
    exe = f"""torchrun \
              --nnodes={nodes} \
              --nproc_per_node={num_gpus} \
              --rdzv-id=$SLURM_JOB_ID \
              --rdzv-backend=c10d \
              --rdzv-endpoint=$RDZV_ENDPOINT"""
if test_run:
    exe = f"CUDA_VISIBLE_DEVICES={','.join(map(str, range(num_gpus)))} {exe}"
    
# use `dataset_dir` instead of `dataset_name` to specify `preprocessed_dir`
# --dataset_name=pile \

## learning rate for pretraining, substituted with finetuning hyperparameters
# --learning_rate 1e-3 \
# --lr_end 1e-4 \
# --adam_epsilon 1e-8 \

## don't need cosine scheduling for finetuning
# --weight_decay 0.01 \
# --lr_scheduler_name linear_warmup_cosine \
# --warmup_ratio 0.06 \

## avoids grad scaling error
# --fp16 \
## for training model from scratch
# --config_overrides="n_positions=1024,n_embd=1024,n_layer=18,n_head=16" \

## added the following
# add_domain_id: for non-pile preprocessed dataset
# do_padding: true for variable size sequences, as in instruction tuning datasets.
# --max_train_samples 1000 \

output_dirname = f'{abbr_model_name}_{abbr_train_file}_baseline'
if test_run:
    output_dirname = 'jpt_'+output_dirname
output_dir = os.path.join(envs['MODEL_OUTPUT_DIR'], job_name, output_dirname)


reference_model_name_or_path = '/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/doremi/results/train_baseline/checkpoint-10'
doremi_options = f"""
--doremi_optimizer=doremiv1 \
--reweight_eta=1 \
--reweight_eps=1e-4 \
--train_domain_weights_tmp_file={os.path.join(output_dir, 'domain_weights')} \
--reweight_domains \
--remove_unused_columns=False \
--reference_model_name_or_path={reference_model_name_or_path} \
"""
doremi_options = ''


cmd = f"""
    {'!cd .. && ' if test_run else ''}{exe}
    doremi/train.py \
    --model_name_or_path={model_name_or_path} \
    --model_type={model_type} \
    --tokenizer_name=gpt2 \
    --do_train \
    --cache_dir={cache_dir} \
    --dataset_dir={dataset_dir} \
    --domain_config_path={domain_config_path} \
    --max_token_length=1024 \
    --per_device_train_batch_size={per_device_train_batch_size} \
    --gradient_accumulation_steps={gradient_accumulation_steps} \
    --dataloader_num_workers=1 \
    --learning_rate=2e-5 \
    --lr_scheduler_type=linear \
    --warmup_ratio=0.03 \
    --weight_decay=0. \
    --max_grad_norm=1.0 \
    --max_steps={max_steps} \
    --evaluation_strategy=no \
    --save_strategy=steps \
    --save_steps={save_steps} \
    --save_total_limit=1 \
    --run_name={job_name} \
    --seed=1111 \
    --logging_strategy=steps \
    --logging_steps=10 \
    --logging_first_step \
    --report_to=tensorboard \
    --optim=adamw_hf \
    --adam_beta1=0.9 \
    --adam_beta2=0.99 \
    {'--gradient_checkpointing' if gradient_checkpointing  else ''} \
    {'--overwrite_output_dir' if overwrite_output_dir else ''} \
    --torch_dtype=float32 \
    --add_domain_id=True \
    --do_padding=True \
    {doremi_options if doremi_options else ''} \
    --output_dir={output_dir} \
"""

if test_run:
    cmds = [x.strip() for x in cmd.split('    ') if x.strip()]
    cmds = ['    '+x if x.startswith('--') else x for x in cmds]
    print()
    print(' \\\n'.join(cmds))
cmd = multiline_to_singleline(cmd)

shell_scripts = shell_scripts_template.format(
    cmd=cmd,
    log_dir=os.getcwd(),
    save_dir=output_dir)
out = submit_job(
    shell_scripts, 
    job_name=job_name, 
    nodes=nodes,
    num_cpus=num_cpus,
    cpu_mem=cpu_mem,
    num_gpus=num_gpus,
    gpu_type=gpu_type,
    test_run=test_run,
    job_duration=job_duration,
)
if not test_run:
    print(out)

Training /gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/results/baselines/gpt2-medium using 6 GPUs, 2 batch size per GPU, 10 gradient accumulation steps,for 1562 max steps.

!cd .. && CUDA_VISIBLE_DEVICES=0,1,2,3,4,5 torchrun \
    --nproc_per_node=6 \
    --master_port=10002 \
doremi/train.py \
    --model_name_or_path=/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/results/baselines/gpt2-medium \
    --model_type=gpt2 \
    --tokenizer_name=gpt2 \
    --do_train \
    --cache_dir=/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache \
    --dataset_dir=/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/data/processed \
    --domain_config_path=/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/configs/humanmix_baseline_50kvocab.json \
    --max_token_length=1024 \
    --per_device_train_batch_size=2 \
    --gradient_accumulation_steps=10 \
    --dataloader_num_workers=1 \
    --learning

In [68]:
!cd .. && CUDA_VISIBLE_DEVICES=0,1,2,3,4,5 torchrun \
    --nproc_per_node=6 \
    --master_port=10002 \
doremi/train.py \
    --model_name_or_path=/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/results/baselines/gpt2-medium \
    --model_type=gpt2 \
    --tokenizer_name=gpt2 \
    --do_train \
    --cache_dir=/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache \
    --dataset_dir=/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/data/processed \
    --domain_config_path=/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/configs/humanmix_baseline_50kvocab.json \
    --max_token_length=1024 \
    --per_device_train_batch_size=2 \
    --gradient_accumulation_steps=10 \
    --dataloader_num_workers=1 \
    --learning_rate=2e-5 \
    --lr_scheduler_type=linear \
    --warmup_ratio=0.03 \
    --weight_decay=0. \
    --max_grad_norm=1.0 \
    --max_steps=1562 \
    --evaluation_strategy=no \
    --save_strategy=steps \
    --save_steps=10 \
    --save_total_limit=1 \
    --run_name=ft1 \
    --seed=1111 \
    --logging_strategy=steps \
    --logging_steps=10 \
    --logging_first_step \
    --report_to=tensorboard \
    --optim=adamw_hf \
    --adam_beta1=0.9 \
    --adam_beta2=0.99 \
    --overwrite_output_dir \
    --torch_dtype=float32 \
    --add_domain_id=True \
    --do_padding=True \
    --fp16 \
    --output_dir=/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/results/ft1/jpt_gpt2-medium_humanmix_baseline

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
[2023-08-02 14:52:02,830] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2023-08-02 14:52:02,867] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2023-08-02 14:52:02,899] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2023-08-02 14:52:02,916] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2023-08-02 14:52:02,919] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2023-08-02 14:52:02,926] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda 

[INFO|modeling_utils.py:2604] 2023-08-02 14:52:11,347 >> loading weights file /gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/results/baselines/gpt2-medium/model.safetensors
[INFO|modeling_utils.py:1176] 2023-08-02 14:52:11,383 >> Instantiating GPT2LMHeadModelDoReMi model under default dtype torch.float32.
[INFO|configuration_utils.py:603] 2023-08-02 14:52:11,384 >> Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.32.0.dev0"
}

08/02/2023 14:52:22 - WARNING - doremi.dataloader - No split used or split directory not found: using same data for all splits.
08/02/2023 14:52:22 - WARNING - datasets.builder - Found cached dataset json (/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache/json/default-20e8d8f5df875937/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)
100%|████████████████████████████████████████████| 1/1 [00:00<00:00

08/02/2023 14:52:25 - WARNING - datasets.arrow_dataset - Loading cached processed dataset at /gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache/json/default-20e8d8f5df875937/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-dac4998a00f3727e_*_of_00008.arrow
08/02/2023 14:52:25 - WARNING - datasets.arrow_dataset - Loading cached processed dataset at /gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache/json/default-20e8d8f5df875937/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-dac4998a00f3727e_*_of_00008.arrow
08/02/2023 14:52:25 - WARNING - datasets.arrow_dataset - Loading cached processed dataset at /gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache/json/default-20e8d8f5df875937/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-dac4998a00f3727e_*_of_00008.arrow
08/02/2023 14:52:25 - WARNING - datasets.arrow_dataset - Loading cached processed datase

08/02/2023 14:52:25 - WARNING - datasets.arrow_dataset - Loading cached processed dataset at /gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache/json/default-a01381664fd2589b/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-dd3397b5949d01cc_*_of_00008.arrow
08/02/2023 14:52:25 - INFO - datasets.arrow_dataset - Concatenating 8 shards
08/02/2023 14:52:25 - WARNING - datasets.builder - Found cached dataset json (/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache/json/default-a01381664fd2589b/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)
100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 306.27it/s]
08/02/2023 14:52:25 - WARNING - datasets.builder - Found cached dataset json (/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache/json/default-a01381664fd2589b/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)
100%|█████████████████████████████████

08/02/2023 14:52:25 - WARNING - datasets.builder - Found cached dataset json (/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache/json/default-d43591dedd2996a1/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)
100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 418.47it/s]
08/02/2023 14:52:25 - WARNING - datasets.builder - Found cached dataset json (/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache/json/default-d43591dedd2996a1/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)
100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 409.36it/s]
08/02/2023 14:52:25 - WARNING - datasets.builder - Found cached dataset json (/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache/json/default-d43591dedd2996a1/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)
100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 422.05it/s]
08/02/2

08/02/2023 14:52:26 - WARNING - datasets.arrow_dataset - Loading cached shuffled indices for dataset at /gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache/json/default-d43591dedd2996a1/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-99401397af038dda.arrow
{'loss': 2.6697, 'learning_rate': 4.2553191489361704e-07, 'epoch': 0.0}         
{'loss': 2.7549, 'learning_rate': 4.255319148936171e-06, 'epoch': 0.01}         
  1%|▏                                      | 10/1562 [01:00<2:31:41,  5.86s/it][INFO|trainer.py:2807] 2023-08-02 14:53:27,614 >> Saving model checkpoint to /gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/results/ft1/jpt_gpt2-medium_humanmix_baseline/checkpoint-10
[INFO|configuration_utils.py:458] 2023-08-02 14:53:27,617 >> Configuration saved in /gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/results/ft1/jpt_gpt2-medium_humanmix_baseline/checkpoint-10/config.json
[INFO|configuration_utils.py:379]


KeyboardInterrupt



In [ ]:

# doremi train baseline on 200k data
# shared: v100_32gb gpu 
# gpt2, nodes=1, num_gpus=6, micro-bsz=8, no-grad-ckpt, fp32, 1hr
# gpt2-medium, nodes=1, num_gpus=6, micro-bsz=4, no-grad-ckpt, fp32, oom
# gpt2-medium, nodes=1, num_gpus=6, micro-gsz=2, no-grad-ckpt, fp32, 2.5hrs
